In [1]:
import sys, time, pickle
import torch

sys.path.insert(0, '../../Utils')
from global_constants import gpu_device

sys.path.insert(0, '../')
from BERT_models import BERT_QNLI_MODEL

from _utils import sample_random_glue_qnli, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info, download_HTML

In [2]:
qnli_data_raw, targets, idxs = sample_random_glue_qnli()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e6d0a9c9270b6822.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c46c5e78d2bda808.arrow


In [3]:
model = BERT_QNLI_MODEL()
tokenizer = model.tokenizer

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import InputXGradient
from captum.attr import visualization 

In [6]:
input_x_gradient  = InputXGradient(model)

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0).to(gpu_device)
    detokenized = [t.replace('#', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()

     #categorizing results
    pred_class = 'No Entailment' if pred_prob < 0.5 else 'Entailment' 
    true_class = 'No Entailment' if target < 0.5 else 'Entailment' 
    
    #attribution algorithm working
    attribution = input_x_gradient.attribute(input_emb)
    word_attributions = attribution.squeeze(0).sum(dim=1)
#     word_attributions = attr_normalizing_func(word_attributions)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'No Entailment' if attr_score < 0.5 else 'Entailment'
    convergence_score = None
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
    print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input_ids=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(qnli_data_raw, targets), start=1):
#     example_1 = "How many times has the South Florida/Miami area hosted the Super Bowl?"
#     example_2 = "The South Florida/Miami area has previously hosted the event 10 times (tied for most with New Orleans), with the most recent one being Super Bowl XLIV in 2010."
    
#     example_1 = "When did the third Digimon series begin?"
#     example_2 = "Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese."
#     datum_raw, target = [example_1, example_2], 1
    print(f'Raw datum: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record])) 


Raw datum: ['What would a teacher assess the levels of a student on?', 'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.']
GT target: 0
word attr tensor([-2.4683e-03, -3.1340e-02, -2.3063e-02,  8.5846e-02, -1.3606e-01,
        -5.6050e-02,  1.7835e-02, -1.2950e-01, -2.3075e-02,  2.8824e-02,
         7.9517e-02, -1.0998e-01, -3.8824e-01,  1.3514e-01,  2.0214e-01,
        -8.6794e-02, -4.3814e-02, -8.9594e-02, -4.3959e-02, -7.5047e-03,
        -1.7923e-01, -5.6481e-05, -8.5562e-02,  1.7122e-01,  4.3536e-02,
        -2.9848e-01,  9.6469e-02, -8.5471e-03, -6.3962e-02,  2.2039e-02,
        -1.4393e-01,  3.8258e-02, -5.2077e-02, -2.8391e-03,  9.6105e-02,
        -4.0598e-04,  4.7579e-02,  6.4932e-03, -1.4074e-01,  3.7176e-02,
         1.5523e-01,  1.5875e-02, -5.0587e-02,  1.7638e-01,  7.4474e-03,
        -6.2674e-03,  1.5195

<IPython.core.display.HTML object>
Raw datum: ['What company created Doctor Who?', 'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, short stories, audio books, radio plays, interactive video games, game books, webcasts, DVD extras, and even stage performances.']
GT target: 1
word attr tensor([-1.7057e-01, -2.7380e-01,  1.2487e-01,  6.1822e-02,  5.1403e-02,
        -3.3132e-01, -7.3765e-01, -5.6606e-03,  4.4995e-02,  2.2840e-01,
         1.0656e-02, -9.7153e-02, -3.0301e-02, -4.7757e-02,  1.4598e-02,
         2.5302e-02, -2.3449e-02,  1.9925e-02,  2.3582e-03,  3.9034e-02,
         7.1666e-02, -2.7007e-02,  2.8336e-02, -5.9740e-02,  1.0856e-02,
         4.5907e-02, -3.5246e-02,  4.0771e-02, -1.3713e-02, -4.4710e-02,
         3.0427e-02, -1.4392e-02,  8.0305e-02,  1.3219e-01, -3.2441e-02,
 

<IPython.core.display.HTML object>
Raw datum: ['What was the name of the Media Day event for Super Bowl 50?', "The game's media day, which was typically held on the Tuesday afternoon prior to the game, was moved to the Monday evening and re-branded as Super Bowl Opening Night."]
GT target: 1
word attr tensor([-0.2020, -0.2829,  0.0656, -0.1091,  0.0100, -0.1244, -0.0514,  0.1761,
         0.0338, -0.1339,  0.0546, -0.0461, -0.0038,  0.2178, -0.2719,  0.2178,
         0.4261, -0.1660, -0.0421,  0.0157, -0.0588,  0.2790,  0.0217,  0.0189,
        -0.0676, -0.0232,  0.1329,  0.0023,  0.0173, -0.0405,  0.0845, -0.0151,
         0.1170, -0.0334, -0.0406, -0.0522,  0.0141, -0.0215, -0.0108,  0.0083,
         0.0238,  0.0175,  0.0066,  0.0165,  0.2044,  0.0414,  0.3047, -0.0781,
         0.2325, -0.0060, -0.0011,  0.0497,  0.2721, -0.0341], device='cuda:0',
       grad_fn=<DivBackward0>)
conti attr [tensor(-0.2020, device='cuda:0', grad_fn=<UnbindBackward0>), tensor(-0.2829, device='cuda:0', 

<IPython.core.display.HTML object>
Raw datum: ['How many Doctor Who soundtracks have been released since 2005?', 'The fourth was released on 4 October 2010 as a two disc special edition and contained music from the 2008–2010 specials (The Next Doctor to End of Time Part 2).']
GT target: 0
word attr tensor([-1.9108e-01, -1.7691e-01,  8.3361e-02,  4.7193e-02, -7.7874e-02,
        -2.2740e-01,  6.2765e-02,  1.6235e-01,  2.0815e-02, -1.2046e-01,
         9.1304e-02, -2.1883e-01, -1.3366e-01, -4.5890e-01,  5.2475e-01,
         7.2280e-02,  5.2501e-03, -1.3569e-01,  7.3141e-03, -1.1589e-02,
         1.1378e-02,  3.4431e-02,  4.6928e-02,  3.9236e-02,  8.7281e-02,
         1.7245e-04,  5.6957e-02,  1.5352e-01,  8.5024e-03,  6.9581e-02,
        -2.0457e-01,  1.3358e-02, -2.1178e-02, -3.7913e-02,  1.0332e-01,
         1.6623e-02, -1.6678e-01, -2.0072e-02,  1.4166e-01,  2.5372e-01,
        -6.2432e-02,  3.3829e-02,  5.0872e-02, -1.1545e-02,  1.7290e-02,
        -1.8756e-02,  4.5824e-02,  8.4401e-

<IPython.core.display.HTML object>
Raw datum: ["What is the name of the country's longest continuously running student film society?", 'Students at the University of Chicago run over 400 clubs and organizations known as Recognized Student Organizations (RSOs).']
GT target: 0
word attr tensor([ 0.0126, -0.3132, -0.0695, -0.0810,  0.0227, -0.0198,  0.0934, -0.0121,
        -0.1446,  0.1678,  0.2868,  0.1358, -0.0951,  0.2214,  0.2305, -0.0305,
         0.0703,  0.1554,  0.0117,  0.0164, -0.1320,  0.0468,  0.1543, -0.2606,
        -0.3230,  0.1233, -0.1689, -0.2971,  0.0040,  0.0560,  0.0109,  0.0253,
         0.0285,  0.1939,  0.2393, -0.0639, -0.1067,  0.0162,  0.1622, -0.2350,
         0.1876,  0.0972], device='cuda:0', grad_fn=<DivBackward0>)
conti attr [tensor(0.0126, device='cuda:0', grad_fn=<UnbindBackward0>), tensor(-0.3132, device='cuda:0', grad_fn=<UnbindBackward0>), tensor(-0.0695, device='cuda:0', grad_fn=<UnbindBackward0>), tensor(-0.0810, device='cuda:0', grad_fn=<UnbindBack

...

<IPython.core.display.HTML object>
Raw datum: ['Who did Britain exploit in India?', 'Although a substantial number of colonies had been designed to provide economic profit and to ship resources to home ports in the seventeenth and eighteenth centuries, Fieldhouse suggests that in the nineteenth and twentieth centuries in places such as Africa and Asia, this idea is not necessarily valid:']
GT target: 0
word attr tensor([-3.6541e-02, -1.7885e-02, -8.5177e-02, -4.4000e-03, -1.2685e-02,
        -2.6675e-02, -1.8557e-01, -3.5055e-01,  3.4683e-01,  3.5934e-01,
        -1.0337e-01, -7.0243e-02,  3.0363e-03, -4.0789e-02, -2.5609e-02,
         4.5579e-02, -2.2030e-02,  9.5143e-05, -3.9767e-02, -3.9843e-03,
        -3.9355e-02,  9.6562e-02, -2.4047e-02, -2.8292e-03,  2.4544e-02,
         3.1649e-02, -2.9933e-02, -7.7694e-03, -3.1601e-02, -4.4408e-02,
         4.7799e-02, -5.0609e-02,  2.6368e-02,  1.0798e-01,  1.8693e-02,
         2.5499e-02, -9.8141e-02, -4.0911e-02, -2.9776e-02,  9.9172e-02,


<IPython.core.display.HTML object>
Raw datum: ['What popular environmentalist is also a university alumni member?', 'In science, alumni include astronomers Carl Sagan, a prominent contributor to the scientific research of extraterrestrial life, and Edwin Hubble, known for "Hubble\'s Law", NASA astronaut John M. Grunsfeld, geneticist James Watson, best known as one of the co-discoverers of the structure of DNA, experimental physicist Luis Alvarez, popular environmentalist David Suzuki, balloonist Jeannette Piccard, biologists Ernest Everett']
GT target: 1
word attr tensor([ 0.0657, -0.0958,  0.0485,  0.0011,  0.1158,  0.0301, -0.0571,  0.0166,
         0.0190,  0.1208,  0.0955, -0.4183,  0.1129,  0.1966, -0.0511,  0.0316,
        -0.0916,  0.2298,  0.0602, -0.0824, -0.0843,  0.0123, -0.0319, -0.0756,
        -0.0505,  0.0344,  0.0935, -0.0148, -0.0266, -0.0050,  0.0094, -0.0232,
         0.0074, -0.0190,  0.0198, -0.0054, -0.0143, -0.0253, -0.0190, -0.0181,
         0.0410, -0.0697,  0.

<IPython.core.display.HTML object>
Raw datum: ['What publication did Philip Howard work for?', 'Responding to the findings of the survey in The Times newspaper, journalist Philip Howard maintained that, "to compare the violence of Dr Who, sired by a horse-laugh out of a nightmare, with the more realistic violence of other television series, where actors who look like human beings bleed paint that looks like blood, is like comparing Monopoly with the property market in London: both are fantasies, but one is meant to be taken seriously."']
GT target: 1
word attr tensor([-0.1491, -0.1364, -0.0582,  0.0584, -0.1525,  0.1899, -0.0835,  0.0986,
        -0.2427,  0.2560,  0.2106, -0.0625, -0.0102,  0.0391, -0.2448,  0.0287,
         0.0024, -0.0760,  0.3384, -0.1034,  0.2044,  0.2851,  0.1278,  0.0172,
        -0.1430,  0.1527, -0.0713, -0.0336,  0.0761,  0.0849, -0.0311, -0.0155,
         0.0549, -0.0270,  0.0808,  0.0201, -0.0503,  0.0556, -0.1277,  0.0109,
        -0.0241, -0.0700,  0.0426

<IPython.core.display.HTML object>
Raw datum: ['If the apparant force of two fermions is attractive, what is the spin function?', 'If two identical fermions (e.g. electrons) have a symmetric spin function (e.g. parallel spins) the spatial variables must be antisymmetric (i.e. they exclude each other from their places much as if there was a repulsive force), and vice versa, i.e. for antiparallel spins the position variables must be symmetric (i.e. the apparent force must be attractive).']
GT target: 1
word attr tensor([ 1.8835e-01, -7.3944e-02, -2.9347e-02, -6.8511e-02,  6.0602e-03,
        -4.9025e-02,  3.1911e-02, -7.9235e-02, -1.8376e-02, -5.6219e-04,
         1.2540e-02,  2.2602e-02,  1.5829e-01,  4.7460e-02, -1.0607e-01,
        -2.1238e-01,  5.9499e-02, -2.4640e-02, -2.4201e-02,  9.3849e-02,
        -1.2835e-01,  2.1887e-01,  1.8123e-01, -9.6763e-02, -1.0319e-01,
        -7.6532e-02,  2.7614e-02, -1.6905e-02,  1.2554e-02,  3.3259e-02,
         4.0247e-02, -9.7427e-03,  6.8936e-02,

<IPython.core.display.HTML object>


In [9]:
file_name_base = 'Input_X_Gradients'

In [10]:
save_info(idxs, qnli_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, 
          fname=f'../MethodOutputs/{file_name_base}_out.pkl')

{'indices': [4013,
  2281,
  2771,
  5109,
  1852,
  2016,
  642,
  3912,
  4427,
  2330,
  4581,
  2611,
  5186,
  2700,
  4920,
  4335,
  1246,
  2811,
  5290,
  5385,
  4774,
  653,
  2816,
  916,
  3297,
  3459,
  1475,
  1287,
  4548,
  1476,
  2328,
  2208,
  2772,
  5301,
  869,
  4968,
  4456,
  4711,
  1929,
  634,
  3774,
  5389,
  497,
  4901,
  4602,
  831,
  1292,
  1087,
  3159,
  3492],
 'raw_data': [['What would a teacher assess the levels of a student on?',
   'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.'],
  ['What company created Doctor Who?',
   'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, sho

In [11]:
download_HTML(cur_file_name=f'{file_name_base}_QNLI_BERT.ipynb',
              out_file_name=f'{file_name_base}_QNLI_BERT.html')